In [ ]:
# #!/usr/bin/python

import sys
import os
import numpy as np
import random
from random import randint
from random import shuffle
import time
import json
import pickle

import tensorflow as tf
from tensorflow.contrib import rnn

# local packages 
from utils_libs import *
from utils_training import *
# from utils_inference import *
# from mixture_models import *
from pipeline import *

# ------ GPU environment
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ------ set-up
para_train = {}

# -- path
para_train['arg_py'] = "market1_tar10_len10"
para_train['path_data'] = "../../datasets/bitcoin/market1_tar10_len10/"
para_train['path_model'] = "../../results/volume/m1_t10_1/"
para_train['path_py'] = "../../results/volume/" + para_train['arg_py']
para_train['path_log_error'] = "../../results/volume/log_" + para_train['arg_py'] + ".txt"

# -- data
para_train['para_num_source'] = 4

# -- model
para_train['para_distr_type'] = 'log_normal_logOpt_linearComb' # "log_normal_logOpt_linearComb", 'normal'
para_train['para_var_type'] = "exp" # square, exp
para_train['para_share_type_gate'] = "no_share" # no_share, share, mix
para_train['para_model_type'] = 'linear'

para_train['para_bool_bias_in_mean'] = True
para_train['para_bool_bias_in_var'] = True
para_train['para_bool_bias_in_gate'] = True

if para_train['para_model_type'] == 'rnn':
    para_train['para_x_src_padding'] = False
    para_train['para_add_common_factor'] = False
    para_train['para_common_factor_type'] = "pool" if para_train['para_add_common_factor'] == True else ""
    
elif para_train['para_model_type'] == 'linear':
    para_train['para_x_src_padding'] = True
    para_train['para_add_common_factor'] = False
    para_train['para_common_factor_type'] = "factor" if para_train['para_add_common_factor'] == True else ""

para_train['para_bool_target_seperate'] = False # [Note] if yes, the last source corresponds to the auto-regressive target variable

# -- training
para_train['para_n_epoch'] = 90 # [Note] for sg_mcmc family, "para_n_epoch" could be set to higher values
para_train['para_burn_in_epoch'] = 85
para_train['para_vali_snapshot_num'] = max(1, int(0.05*para_train['para_n_epoch']))
para_train['para_test_snapshot_num'] = 10

para_train['para_hpara_search'] = "random" # random, grid 
para_train['para_hpara_train_trial_num'] = 30
para_train['para_hpara_retrain_num'] = 10
para_train['para_hpara_ensemble_trial_num'] = 3

# -- optimization
para_train['para_loss_type'] = "heter_lk_inv" # "heter_lk_inv"
para_train['para_optimizer'] = "adam" # RMSprop, adam, sgd, adamW, sg_mcmc_RMSprop, sg_mcmc_adam
para_train['para_optimizer_lr_decay_epoch'] = 10 # after the warm-up
para_train['para_optimizer_lr_warmup_epoch'] = max(1, int(0.1*para_train['para_n_epoch']))

para_train['para_early_stop_bool'] = False
para_train['para_early_stop_window'] = 0

para_train['para_validation_metric'] = 'nnllk'
para_train['para_metric_map'] = {'rmse':0, 'mae':1, 'mape':2, 'nnllk':3}

# -- regularization
para_train['para_regu_mean'] = True
para_train['para_regu_var'] = True
para_train['para_regu_gate'] = False

# ----- hpara: hyper parameter ranges

para_hpara_range = {}
para_hpara_range['random'] = {}
para_hpara_range['random']['linear'] = {}
para_hpara_range['random']['rnn'] = {}

# -- linear
if para_train['para_add_common_factor'] == True:
    para_hpara_range['random']['linear']['factor_size'] = [10, 10]
para_hpara_range['random']['linear']['lr'] = [1e-4, 5e-4]
para_hpara_range['random']['linear']['batch_size'] = [10, 300]
# source-wise
# tar1
# para_hpara_range['random']['linear']['l2_mean'] = [1e-1, 5e+1]
# para_hpara_range['random']['linear']['l2_var']  = [1e-1, 5e+1]
# # tar5
# para_hpara_range['random']['linear']['l2_mean'] = [1e-1, 1e+1]
# para_hpara_range['random']['linear']['l2_var']  = [1e-1, 1e+1]
# tar10
para_hpara_range['random']['linear']['l2_mean'] = [1e-1, 5e-0]
para_hpara_range['random']['linear']['l2_var']  = [1e-1, 5e-0]

# # -- rnn
# # source-wise
# para_hpara_range['random']['rnn']['rnn_size'] =  [16, 16]
# para_hpara_range['random']['rnn']['dense_num'] = [0, 3] # inproper value leads to non-convergence in training

# para_hpara_range['random']['rnn']['lr'] = [0.001, 0.001]
# para_hpara_range['random']['rnn']['batch_size'] = [100, 140]

# # source-wise
# para_hpara_range['random']['rnn']['l2_mean'] = [1e-7, 1e-3]
# para_hpara_range['random']['rnn']['l2_var'] = [1e-7, 1e-3]
# if para_regu_gate == True:
#     para_hpara_range['random']['linear']['l2_gate'] = [1e-7, 1e-3]
    
# para_hpara_range['random']['rnn']['dropout_keep_prob'] = [0.7, 1.0]
# para_hpara_range['random']['rnn']['max_norm_cons'] = [0.0, 0.0]

# ----- main process  

if __name__ == '__main__':
    
    # --- data
    src_tr_x, tr_y, src_val_x, val_y, src_ts_x, ts_y = prepare_data(para_train)
    
    # --- run the pipeline
    log_setup(path = para_train['path_log_error'], 
              para_train = para_train,
              para_hpara_range = para_hpara_range)
    
    print('\n --- ', para_train)
    
    train_validate_test(src_tr_x,
                        tr_y,
                        src_val_x,
                        val_y,
                        src_ts_x,
                        ts_y,
                        hyper_para_range = para_hpara_range,
                        para_train = para_train)
    

In [ ]:
# -- hyper-para generator 
    if para_hpara_search == "random":        
        hpara_generator = hyper_para_random_search(para_hpara_range[para_hpara_search][para_model_type], 
                                                   para_hpara_train_trial_num)
    elif para_hpara_search == "grid":
        hpara_generator = hyper_para_grid_search(para_hpara_range[para_hpara_search][para_model_type])
            
    # -- begin hyper-para search
    hpara_log = []
    
    # sample one set-up of hyper-para
    hpara_dict = hpara_generator.one_trial()
                                                 
    while hpara_dict != None:
        
        tr_dict = training_para_gen(shape_x_dict = shape_tr_x_dict, 
                                    hpara_dict = hpara_dict)
        # hp_: hyper-parameter
        # hp_step_error: [[step, train_metric, val_metric, epoch]]
        hp_step_error, hp_epoch_time = training_validating(src_tr_x,
                                                           tr_y,
                                                           src_val_x,
                                                           val_y,
                                                           dim_x = para_dim_x,
                                                           steps_x = para_steps_x,
                                                           hyper_para_dict = hpara_dict,
                                                           training_dict = tr_dict,
                                                           retrain_bool = False,
                                                           retrain_top_steps = [],
                                                           retrain_bayes_steps = [],
                                                           retrain_idx = 0,
                                                           random_seed = 1)
        
        #[ dict{lr, batch, l2, ..., burn_in_steps}, [[step, tr_metric, val_metric, epoch]] ]
        hpara_dict["burn_in_steps"] = para_burn_in_epoch # tr_dict["batch_per_epoch"] - 1
        hpara_log.append([hpara_dict, hp_step_error])
        
        # -- prepare for the next trial
        
        # sample the next hyper-para
        hpara_dict = hpara_generator.one_trial()
        
        # -- logging
        log_train_val_performance(path_log_error,
                                  hpara = hpara_log[-1][0],
                                  hpara_error = hpara_log[-1][1][0],
                                  train_time = hp_epoch_time)
        # NAN loss exception
        log_null_loss_exception(hp_step_error, 
                                path_log_error)
        
        print('\n Validation performance under the hyper-parameters: \n', hpara_log[-1][0], hpara_log[-1][1][0])
        print('\n Training time: \n', hp_epoch_time, '\n')
        
    # ------ re-train
    #save all epoches in re-training, then select snapshots
    
    # best hyper-para
    best_hpara, _, _, _, _ = hyper_para_selection(hpara_log, 
                                                  val_snapshot_num = para_vali_snapshot_num, 
                                                  test_snapshot_num = para_test_snapshot_num,
                                                  metric_idx = para_metric_map[para_validation_metric])
    retrain_hpara_steps = []
    retrain_hpara_step_error = []
    retrain_random_seeds = [1] + [randint(0, 1000) for _ in range(para_hpara_retrain_num-1)]
    
    for tmp_retrain_id in range(para_hpara_retrain_num):
        
        tr_dict = training_para_gen(shape_x_dict = shape_tr_x_dict,
                                    hpara_dict = best_hpara)
        
        step_error, _ = training_validating(src_tr_x,
                                            tr_y,
                                            src_val_x,
                                            val_y,
                                            dim_x = para_dim_x,
                                            steps_x = para_steps_x,
                                            hyper_para_dict = best_hpara,
                                            training_dict = tr_dict,
                                            retrain_bool = True,
                                            retrain_top_steps = list(range(para_n_epoch)), #top_steps,
                                            retrain_bayes_steps = list(range(para_n_epoch)), #bayes_steps,
                                            retrain_idx = tmp_retrain_id,
                                            random_seed = retrain_random_seeds[tmp_retrain_id])
        
        top_steps, bayes_steps, top_steps_features, bayes_steps_features, val_error, step_error_pairs = snapshot_selection(train_log = step_error,
                                                                                                                           snapshot_num = para_test_snapshot_num,
                                                                                                                           total_step_num = para_n_epoch,
                                                                                                                           metric_idx = para_metric_map[para_validation_metric],
                                                                                                                           val_snapshot_num = para_vali_snapshot_num)
        if len(top_steps) != 0:
            retrain_hpara_steps.append([top_steps, bayes_steps, top_steps_features, bayes_steps_features, tmp_retrain_id, val_error])
            retrain_hpara_step_error.append([step_error_pairs, tmp_retrain_id])
        
        log_val_hyper_para(path = path_log_error,
                           hpara_tuple = [best_hpara, top_steps],
                           error_tuple = step_error[0], 
                           log_string = "-- " + str(tmp_retrain_id))
    
        print('\n----- Retrain hyper-parameters: ', best_hpara, top_steps, '\n')
        print('\n----- Retrain validation performance: ', step_error[0], '\n')
    
    sort_retrain_hpara_steps = sorted(retrain_hpara_steps, 
                                      key = lambda x:x[-1])
    
    log_test_performance(path = path_log_error, 
                         error_tuple = [i[-2:] for i in sort_retrain_hpara_steps], 
                         ensemble_str = "Retrain Ids and Vali. Errors: ")
    
    log_test_performance(path = path_log_error, 
                         error_tuple = [i[-2:] for i in sort_retrain_hpara_steps[:para_hpara_ensemble_num]], 
                         ensemble_str = "Retrain Ids for ensemble: ")
    
    # ------ testing
    # error tuple: [rmse, mae, mape, nnllk]
    # py_tuple
    
    # -- one snapshot from one retrain
    error_tuple, py_tuple = testing(retrain_snapshots = [sort_retrain_hpara_steps[0][0][:1]],
                                    retrain_ids = [ sort_retrain_hpara_steps[0][-2] ],
                                    xts = src_ts_x, 
                                    yts = ts_y, 
                                    file_path = path_model, 
                                    bool_instance_eval = True,
                                    loss_type = para_loss_type,
                                    num_src = len(src_val_x),
                                    snapshot_features = [],
                                    hpara_dict = best_hpara, 
                                    para_model_type = para_model_type, 
                                    para_loss_type = para_loss_type)
    log_test_performance(path = path_log_error, 
                         error_tuple = [error_tuple], 
                         ensemble_str = "One-shot-one-retrain")
    # dump predictions
    pickle.dump(py_tuple, open(path_py + "_one_one" + ".p", "wb"))
    
    # -- one snapshot from multi retrain
    error_tuple, py_tuple = testing(retrain_snapshots = [tmp_steps[0][:1] for tmp_steps in sort_retrain_hpara_steps], 
                                    retrain_ids = [i[-2] for i in sort_retrain_hpara_steps[:para_hpara_ensemble_num]],
                                    xts = src_ts_x,
                                    yts = ts_y, 
                                    file_path = path_model,
                                    bool_instance_eval = True,
                                    loss_type = para_loss_type,
                                    num_src = len(src_ts_x), 
                                    snapshot_features = [], 
                                    hpara_dict = best_hpara, 
                                    para_model_type = para_model_type, 
                                    para_loss_type = para_loss_type)
    log_test_performance(path = path_log_error, 
                         error_tuple = [error_tuple], 
                         ensemble_str = "One-shot-multi-retrain")
    # dump predictions
    pickle.dump(py_tuple, open(path_py + "_one_multi" + ".p", "wb"))
    
    # -- top snapshots from one retrain
    error_tuple, py_tuple = testing(retrain_snapshots = [sort_retrain_hpara_steps[0][0]], 
                                    retrain_ids = [ sort_retrain_hpara_steps[0][-2] ], 
                                    xts = src_ts_x, 
                                    yts = ts_y, 
                                    file_path = path_model,
                                    bool_instance_eval = True, 
                                    loss_type = para_loss_type, 
                                    num_src = len(src_ts_x), 
                                    snapshot_features = [], 
                                    hpara_dict = best_hpara, 
                                    para_model_type = para_model_type, 
                                    para_loss_type = para_loss_type)
    log_test_performance(path = path_log_error,
                         error_tuple = [error_tuple],
                         ensemble_str = "Top-shots-one-retrain")
    # dump predictions
    pickle.dump(py_tuple, open(path_py + "_top_one" + ".p", "wb"))
    
    # -- top snapshots multi retrain
    error_tuple, py_tuple = testing(retrain_snapshots = [tmp_steps[0] for tmp_steps in sort_retrain_hpara_steps], 
                                    retrain_ids = [i[-2] for i in sort_retrain_hpara_steps[:para_hpara_ensemble_num]], 
                                    xts = src_ts_x,
                                    yts = ts_y,
                                    file_path = path_model,
                                    bool_instance_eval = True,
                                    loss_type = para_loss_type,
                                    num_src = len(src_ts_x), 
                                    snapshot_features = [], 
                                    hpara_dict = best_hpara, 
                                    para_model_type = para_model_type, 
                                    para_loss_type = para_loss_type)
    log_test_performance(path = path_log_error, 
                         error_tuple = [error_tuple], 
                         ensemble_str = "Top-shots-multi-retrain")
    # dump predictions
    pickle.dump(py_tuple, open(path_py + "_top_multi" + ".p", "wb"))
    
    # -- bayesian snapshots one retrain
    error_tuple, py_tuple = testing(retrain_snapshots = [sort_retrain_hpara_steps[0][1]], 
                                    retrain_ids = [ sort_retrain_hpara_steps[0][-2] ], 
                                    xts = src_ts_x, 
                                    yts = ts_y,
                                    file_path = path_model, 
                                    bool_instance_eval = True, 
                                    loss_type = para_loss_type, 
                                    num_src = len(src_ts_x), 
                                    snapshot_features = [], 
                                    hpara_dict = best_hpara, 
                                    para_model_type = para_model_type, 
                                    para_loss_type = para_loss_type)
    log_test_performance(path = path_log_error, 
                         error_tuple = [error_tuple], 
                         ensemble_str = "Bayesian-one-retrain")
    # dump predictions
    pickle.dump(py_tuple, open(path_py + "_bayes_one" + ".p", "wb"))
    
    # -- bayesian snapshots multi retrain
    error_tuple, py_tuple = testing(retrain_snapshots = [tmp_steps[1] for tmp_steps in sort_retrain_hpara_steps],
                                    retrain_ids = [i[-2] for i in sort_retrain_hpara_steps[:para_hpara_ensemble_num]],
                                    xts = src_ts_x,
                                    yts = ts_y,
                                    file_path = path_model,
                                    bool_instance_eval = True,
                                    loss_type = para_loss_type,
                                    num_src = len(src_ts_x),
                                    snapshot_features = [],
                                    hpara_dict = best_hpara, 
                                    para_model_type = para_model_type, 
                                    para_loss_type = para_loss_type)
    log_test_performance(path = path_log_error,
                         error_tuple = [error_tuple],
                         ensemble_str = "Bayesian-multi-retrain")
    # dump predictions
    pickle.dump(py_tuple, open(path_py + "_bayes_multi" + ".p", "wb"))
    
    # -- global top1 and topK steps
    
    retrain_ids, retrain_id_steps = global_top_steps_multi_retrain(retrain_step_error = retrain_hpara_step_error, 
                                                                   num_step = int(para_test_snapshot_num*para_hpara_ensemble_num))    
#     log_test_performance(path = path_log_error, 
#                          error_tuple = [retrain_ids, retrain_id_steps], 
#                          ensemble_str = "Global-top-steps: ")
    
    error_tuple, py_tuple = testing(retrain_snapshots = retrain_id_steps, 
                                    retrain_ids = retrain_ids,
                                    xts = src_ts_x,
                                    yts = ts_y, 
                                    file_path = path_model,
                                    bool_instance_eval = True,
                                    loss_type = para_loss_type,
                                    num_src = len(src_ts_x), 
                                    snapshot_features = [], 
                                    hpara_dict = best_hpara, 
                                    para_model_type = para_model_type, 
                                    para_loss_type = para_loss_type)
    log_test_performance(path = path_log_error, 
                         error_tuple = [error_tuple], 
                         ensemble_str = "Global-top-steps-multi-retrain ")
    # dump predictions
    pickle.dump(py_tuple, open(path_py + "_global" + ".p", "wb"))
    